Code to extract and save $\Delta F / F$ for whole making whole brain maps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
from pathlib import Path
import pickle
#import re

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import janelia_core
from janelia_core.dataprocessing.dataset import ROIDataset
#from janelia_core.utils.data_saving import append_ts

#from keller_zlatic_vnc.data_processing import extract_transitions
from keller_zlatic_vnc.data_processing import calc_dff
from keller_zlatic_vnc.data_processing import match_annotation_subject_to_volume_subject
from keller_zlatic_vnc.data_processing import read_raw_transitions_from_excel
#from keller_zlatic_vnc.data_processing import recode_beh
from keller_zlatic_vnc.data_processing import down_select_events
from keller_zlatic_vnc.data_processing import find_before_and_after_events
from keller_zlatic_vnc.data_processing import generate_standard_id_for_full_annots
from keller_zlatic_vnc.data_processing import read_full_annotations
from keller_zlatic_vnc.data_processing import whole_brain_extract_dff_with_annotations

## Parameters go here

In [3]:
ps = dict()

# The file specifying which subjects we should include in the analysis
ps['subject_file'] = r'Z:\bishoplab\projects\keller_drive\keller_vnc\data\single_cell\subjects.csv'

# Location of folders containing annotations
ps['a4_annot_folder'] = r'Z:\bishoplab\projects\keller_drive\keller_vnc\data\full_annotations\behavior_csv_cl_A4'
ps['a9_annot_folder'] = r'Z:\bishoplab\projects\keller_drive\keller_vnc\data\full_annotations\behavior_csv_cl_A9'

# Location of file containing Chen's annotations - we use this to filter down to only good stimulus events
ps['chen_file'] = r'Z:\bishoplab\projects\keller_drive\keller_vnc\data\extracted_dff_v2\transition_list_CW_11202021.xlsx'

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'Z:\bishoplab\projects\keller_drive\keller_vnc\data\experiment_data_locations.xlsx'

# Base folder where datasets are stored 
ps['dataset_base_folder'] =r'W:\\SV4'

# Subfolder containing the dataset for each subject
ps['dataset_folder'] = 'extracted'

# Parameters for declaring preceeding and succeeding quiet behaviors
ps['pre_q_th'] = 50
ps['succ_q_th'] = 9

# Parameters for determing the location of the marked preceeding and succeeding quiet events
ps['pre_q_event_l'] = 3 # Event length for the preceeding quiet event
ps['succ_q_event_l'] = 3 # Event length for the succeeding quiet event

# Data to calculate Delta F/F for in each dataset
ps['f_ts_str'] = 'f_1_5_5'
ps['bl_ts_str'] = 'bl_1_5_5_long'

# Parameters for calculating Delta F/F
ps['dff_calc_params'] = dict()
ps['dff_calc_params']['background'] = 100
ps['dff_calc_params']['ep'] = 20

# Parameters for where windows are placed
ps['dff_window_type'] = 'start_end_aligned'
ps['dff_window_ref'] = 'start'
ps['dff_window_offset'] = 0
ps['dff_window_length'] = None
ps['dff_window_end_ref'] = 'end'
ps['dff_window_end_offset'] = 1

# Specify where we save our results
ps['save_folder'] = r'Z:\bishoplab\projects\keller_drive\keller_vnc\results\new_whole_brain_stats\1_5_5'
ps['save_str'] = 'decision_dep_1_5_5'


## Read in excel file specifying location of datasets

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Find stimulus events for all subjects

In [5]:
# Get list of subjects we have annotations for

a4_file_paths = glob.glob(str(Path(ps['a4_annot_folder']) / '*.csv'))
a9_file_paths = glob.glob(str(Path(ps['a9_annot_folder']) / '*.csv'))

n_annot_files = len(a4_file_paths) + len(a9_file_paths)
a4_files = np.zeros(n_annot_files, dtype=np.bool)
a4_files[0:len(a4_file_paths)] = True

annot_file_paths = a4_file_paths + a9_file_paths

annot_file_names = [Path(p).name for p in annot_file_paths]
annot_subjs = [generate_standard_id_for_full_annots(fn) for fn in annot_file_names]


C:\Users\SiMView\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


## Get stimulus events for each subject

In [6]:
subj_events = pd.DataFrame()

for ind, subj in enumerate(annot_subjs):

    # Load the annotations for this subject
    tbl = read_full_annotations(annot_file_paths[ind])

    # Pull out stimulus events for this subject, noting what comes before and after
    stim_tbl = copy.deepcopy(tbl[tbl['beh'] == 'S'])
    stim_tbl.insert(0, 'subject_id', subj)
    stim_tbl.insert(1, 'event_id', range(stim_tbl.shape[0]))
    if a4_files[ind] == True:
        stim_tbl.insert(2, 'manipulation_tgt', 'A4')
    else:
        stim_tbl.insert(2, 'manipulation_tgt', 'A9')
    before_after_tbl = find_before_and_after_events(events=stim_tbl, all_events=tbl)
    stim_annots = pd.concat([stim_tbl, before_after_tbl], axis=1)
    subj_events = subj_events.append(stim_annots, ignore_index=True)


## Get rid of any events where we could not classify the type of preceeding or succeeding behavior

In [7]:
subj_events = subj_events.dropna()

## Get rid of any stimulus events which are not also in Chen's annotations

We do this because some of the stimulus events in the full annotations (Nadine's annotations) should be removed because of artefacts. Chen's annotations only include the stimulus events we should analyze.

In [8]:
chen_events = read_raw_transitions_from_excel(file=ps['chen_file'])
chen_events = chen_events.rename(columns={'Manipulation Start': 'start', 'Manipulation End': 'end'})
subj_events = down_select_events(tbl_1=subj_events, tbl_2=chen_events)

Removing event from: 4359-4364 for subject CW_17-08-23-L4
Removing event from: 6004-6009 for subject CW_17-08-23-L4
Removing event from: 267-272 for subject CW_17-09-01-L3


## Mark preceeding and succeeding quiet events

In [9]:
delta_before = subj_events['start'] - subj_events['beh_before_end']
delta_after = subj_events['beh_after_start'] - subj_events['end']

before_quiet_inds = delta_before > ps['pre_q_th']
after_quiet_inds = delta_after > ps['succ_q_th']

subj_events.loc[before_quiet_inds, 'beh_before'] = 'Q'
subj_events.loc[after_quiet_inds, 'beh_after'] = 'Q'


## Mark the start and stop of the marked quiet events

In [10]:
new_before_start = (np.ceil((subj_events[before_quiet_inds]['start'] -
                             subj_events[before_quiet_inds]['beh_before_end']) / 2) +
                             subj_events[before_quiet_inds]['beh_before_end'])

new_before_end = new_before_start + ps['pre_q_event_l'] - 1  # Minus 1 for inclusive indexing
subj_events.loc[before_quiet_inds, 'beh_before_start'] = new_before_start
subj_events.loc[before_quiet_inds, 'beh_before_end'] = new_before_end

new_after_start = (np.ceil((subj_events[after_quiet_inds]['beh_after_start'] -
                            subj_events[after_quiet_inds]['end']) / 2) +
                            subj_events[after_quiet_inds]['end'])
new_after_end = new_after_start + ps['succ_q_event_l'] - 1  # Minus 1 for inclusive indexing
subj_events.loc[after_quiet_inds, 'beh_after_start'] = new_after_start
subj_events.loc[after_quiet_inds, 'beh_after_end'] = new_after_end

## Extract dff along with behavior information from each dataset

In [11]:
analysis_subjects = subj_events['subject_id'].unique()
n_datasets = len(data_locs)
n_analysis_subjs = len(analysis_subjects)
matched_analysis_subjs = np.zeros(n_analysis_subjs)
matched_data_subjs = np.zeros(n_datasets)

subject_event_data = [None]*n_datasets

for d_i in range(n_datasets):
    
    # ===============================================================================
    # Find the appropriate annotations for this experiment
    data_main_folder = data_locs['Main folder'][d_i]
    data_sub_folder = data_locs['Subfolder'][d_i]

    # Parse the subject from the subfolder string
    match_ind = match_annotation_subject_to_volume_subject(data_main_folder, data_sub_folder, analysis_subjects)
    
    if match_ind is not None:
        match_ind = match_ind.item()
        
        
        # Prevent any double matching
        if matched_analysis_subjs[match_ind] == True:
            raise(RuntimeError('Found a transition subject we already matched to: d_i=' + str(d_i)))
        if matched_data_subjs[d_i] == True:
            raise(RuntimeError('Found a volume subject we already matched to.'))
            
        matched_analysis_subjs[match_ind] = True
        matched_data_subjs[d_i] = True
         
        # ===============================================================================
        # Now that we know we have a unique match, we do the rest of our processing

        # Read in the data for this subject
        dataset_path = (Path(ps['dataset_base_folder']) / data_main_folder / data_sub_folder / 
                        Path(ps['dataset_folder']))
        dataset_file = glob.glob(str(dataset_path/'*.pkl'))[0]

        # Load the dataset
        with open(dataset_file, 'rb') as f:
            dataset = ROIDataset.from_dict(pickle.load(f))
        
        # Update the NDArrayHandlers within the dataset object, so they know where to load data from
        for ts_label in dataset.ts_data.keys():
            ts_vls = dataset.ts_data[ts_label]['vls']
            if isinstance(ts_vls, janelia_core.fileio.data_handlers.NDArrayHandler):
                ts_vls.folder = str(dataset_path / Path(ts_vls.folder).name)
        
        # Calculate dff
        f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
        b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]
        dff = calc_dff(f=f, b=b, background=ps['dff_calc_params']['background'], ep=ps['dff_calc_params']['ep'])
        
        # Extract dff for each event
        sample_id = analysis_subjects[match_ind]
        event_rows = subj_events['subject_id'] == sample_id
        sample_events = copy.deepcopy(subj_events[event_rows])
        
        subject_event_data[d_i] = whole_brain_extract_dff_with_annotations(dff=dff,
                                                              event_tbl=sample_events,
                                                              align_col=ps['dff_window_ref'],
                                                              window_type=ps['dff_window_type'],
                                                              ref_offset=ps['dff_window_offset'],
                                                              window_l=ps['dff_window_length'],
                                                              end_align_col=ps['dff_window_end_ref'],
                                                              end_ref_offset=ps['dff_window_end_offset'])
        
        # Give user some feedback
        print('Done processing dataset ' + str(d_i + 1) + ' of ' + str(n_datasets) + '.')

Done processing dataset 1 of 65.
Done processing dataset 2 of 65.
Done processing dataset 3 of 65.
Done processing dataset 4 of 65.
Done processing dataset 5 of 65.
Done processing dataset 6 of 65.
Done processing dataset 7 of 65.
Done processing dataset 8 of 65.
Done processing dataset 9 of 65.


KeyboardInterrupt: 

In [ ]:
subject_event_data = pd.concat(subject_event_data, ignore_index=True)

In [ ]:
# See what subjects we have transitions for but no registerd volumes
ignored_analysis_subjs = analysis_subjects[np.logical_not(matched_analysis_subjs)]

# See what subjects we have registered volumes for but no transitions
ignored_vol_subjs = data_locs[np.logical_not(matched_data_subjs)]

In [ ]:
## Save results
rs = dict()
rs['ps'] = ps
rs['subject_event_data'] = subject_event_data
rs['ignored_analysis_subjs'] = ignored_analysis_subjs
rs['ignored_vol_subjs'] = ignored_vol_subjs

save_name = ps['save_str'] + '.pkl'
save_path = Path(ps['save_folder']) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)
    
print('Saved results to: ' + str(save_path))

In [ ]:
subject_event_data

In [ ]:
subj_events